<a href="https://colab.research.google.com/github/faranak5171/Deep-Learning-projects/blob/master/flower_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torchvision import transforms, datasets, models
from torch import nn, optim

In [2]:
%%capture
! unrar x train.rar train
! unrar x valid.rar valid

In [12]:
%%capture
! unrar x test.rar test

In [13]:
# TODO: Define your transforms for the training, validation, and testing sets
transform = transforms.Compose([transforms.Resize(255),
                                transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

# TODO: Load the datasets with ImageFolder
train_dataset = datasets.ImageFolder('train', transform=transform)
valid_dataset = datasets.ImageFolder('valid', transform=transform)
test_dataset = datasets.ImageFolder('test', transform=transform)

# TODO: Using the image datasets and the trainforms, define the dataloaders
trainloaders = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
validloaders = torch.utils.data.DataLoader(valid_dataset, batch_size=64, shuffle=True)
testloaders = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

In [5]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

In [8]:
# TODO: Build and train your network
model = models.vgg16(pretrained=True)
num_fetures = model.classifier[0].in_features

for param in model.parameters():
    param.requires_grad = False

In [9]:
model.classifier = nn.Sequential(nn.Linear(num_fetures,204),
                                nn.ReLU(),
                                nn.Dropout(p=0.2),
                                nn.Linear(204,102),
                                nn.LogSoftmax(dim=1))

criterion = nn.NLLLoss()
optimizer = optim.SGD(model.classifier.parameters(), lr=0.01)

In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [11]:
epochs = 10
for epoch in range(epochs):
    
    model.train()
    running_loss = 0
    print("Start training the model")
    for data, labels in trainloaders:
        data, labels = data.to(device), labels.to(device)
        optimizer.zero_grad()
        logps = model.forward(data)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    val_loss = 0
    accuracy = 0
    model.eval()
    print("Start Validation the model")
    for data, labels in validloaders:
        data, labels = data.to(device), labels.to(device)
        logps = model.forward(data)
        loss = criterion(logps, labels)
        val_loss += loss.item()
        
        ps = torch.exp(logps)
        top_k, top_class = logps.topk(1,dim=1)
        equal = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equal.type(torch.FloatTensor)).item()
    
    print(f"epochs: {epoch + 1}/{epochs}.. Average Train Loss:{running_loss/len(trainloaders):.3f}.."
            f"Average validation Loss:{val_loss/len(validloaders):.3f}.. Accuracy: {accuracy/len(validloaders):.3f}..")

Start training the model
Start Validation the model
epochs: 1/10.. Average Train Loss:3.055..Average validation Loss:1.863.. Accuracy: 0.570..
Start training the model
Start Validation the model
epochs: 2/10.. Average Train Loss:1.240..Average validation Loss:1.114.. Accuracy: 0.777..
Start training the model
Start Validation the model
epochs: 3/10.. Average Train Loss:0.684..Average validation Loss:0.808.. Accuracy: 0.838..
Start training the model
Start Validation the model
epochs: 4/10.. Average Train Loss:0.434..Average validation Loss:0.673.. Accuracy: 0.861..
Start training the model
Start Validation the model
epochs: 5/10.. Average Train Loss:0.301..Average validation Loss:0.586.. Accuracy: 0.887..
Start training the model
Start Validation the model
epochs: 6/10.. Average Train Loss:0.227..Average validation Loss:0.535.. Accuracy: 0.895..
Start training the model
Start Validation the model
epochs: 7/10.. Average Train Loss:0.176..Average validation Loss:0.516.. Accuracy: 0.901..

In [17]:
accuracy = 0 
for data, labels in testloaders:
  data, labels = data.to(device), labels.to(device)
  logps = model.forward(data)
  ps = torch.exp(logps)
  top_k, top_class = ps.topk(1, dim=1)
  equal = top_class == labels.view(*top_class.shape)
  accuracy += torch.mean(equal.type(torch.FloatTensor)).item()

print(f"Test Accuracy: {accuracy/len(testloaders):.3f}..")

Test Accuracy: 0.878..
